In [ ]:
!pip install mlflow

In [1]:
import mlflow
import os
from transformers.integrations import MLflowCallback
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer
import pandas as pd
import torch
from datasets import Dataset
import transformers
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
from datasets import load_metric
import datasets

In [2]:
os.environ['HF_MLFLOW_LOG_ARTIFACTS'] = "0"
print(os.getenv("HF_MLFLOW_LOG_ARTIFACTS"))

0


In [3]:
metric = load_metric("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

In [4]:
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "https://mlflow-c3dev.concertohealth.ai"))
mlflow.set_experiment(experiment_name="transformer")

<Experiment: artifact_location='s3://ai-engg-ml-models/3', experiment_id='3', lifecycle_stage='active', name='transformer', tags={}>

In [5]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model_name = "bert-base-cased"
model_check_point = "bert-base-cased"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_check_point)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
model.config.label2id = {
    'G': 1,
    "B": 2,
    "N" : 0
}

In [8]:
model.config.id2label = {
    1: "G",
    2: "B",
    0: "N"
}

In [9]:
df = pd.read_csv("./train_small_v4.csv")

In [10]:
dataset = Dataset.from_pandas(df)

In [11]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
encoded_map = tokenized_datasets.map(lambda x: {'label':model.config.label2id[x['label']]})

0ex [00:00, ?ex/s]

In [13]:
training_args = TrainingArguments("test_trainer", 
                                  evaluation_strategy='epoch',
                                  num_train_epochs=10)

In [14]:
trainer = Trainer(model=model, 
                  args=training_args,
                  train_dataset=encoded_map,
                  eval_dataset=encoded_map,
                  compute_metrics=compute_metrics,)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
/home/hnamburi@ConcertoHealth.AI/anaconda3/envs/fai/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 498
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 630


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Epoch,Training Loss,Validation Loss,F1
1,No log,0.604542,0.743324
2,No log,0.290966,0.912871
3,No log,0.226596,0.949840
4,No log,0.140326,0.959898
5,No log,0.089782,0.973784
6,No log,0.069818,0.979902
7,No log,0.053287,0.985926
8,0.300300,0.049494,0.987968


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 498
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 498
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 498
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence.
***** Running Evaluation *****
  Num examples = 498
  Batch size = 8
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequ

In [ ]:
trainer.save_model("buy_sell_v8")